# SPARKLE: Advanced Quantum Computing

Master the most sophisticated quantum computing concepts and techniques!

This notebook covers advanced topics that are at the forefront of quantum computing research and applications. These concepts are essential for understanding real-world quantum systems.

## TARGET: Advanced Topics Covered

1. SECURITY: **Quantum Error Correction** - Protecting quantum information
2. WAVE: **Noise and Decoherence** - Real-world quantum challenges
3. LINK: **Quantum Entanglement Theory** - Deep dive into quantum correlations 
4. DEMO: **Quantum State Tomography** - Measuring unknown quantum states
5. **Quantum Supremacy Circuits** - Cutting-edge quantum demonstrations
6. AI: **Quantum Machine Learning** - AI meets quantum computing

Ready to master advanced quantum computing? Let's explore this in detail. WAVE: 

In [ ]:
from quantumsim import Circuit, Executor
from quantumsim.viz.ascii import ASCIIDrawer
from quantumsim.noise.channels import NoiseChannel
from quantumsim.algorithms.error_correction import ShorCode, SurfaceCode
import numpy as np
import matplotlib.pyplot as plt
from itertools import product

print("SPARKLE: Advanced Quantum Computing")
print("=" * 32)

executor = Executor()

## SECURITY: Section 1: Quantum Error Correction

Quantum information is fragile! Noise and decoherence destroy quantum states. Error correction protects quantum information by encoding logical qubits into multiple physical qubits.

### Three-Qubit Bit-Flip Code

The simplest quantum error correction code protects against single bit-flip errors.

In [ ]:
def three_qubit_bit_flip_code():
 """
 Implement and test the three-qubit bit-flip error correction code
 """
 print("SECURITY: Three-Qubit Bit-Flip Error Correction")
 print("=" * 45)
 
 def encode_logical_zero():
 """Encode logical |0 as |000"""
 circuit = Circuit(3)
 # Logical |0 = |000 (no operations needed)
 return circuit
 
 def encode_logical_one():
 """Encode logical |1 as |111"""
 circuit = Circuit(3)
 circuit.x(0)
 circuit.cx(0, 1)
 circuit.cx(0, 2)
 return circuit
 
 def add_syndrome_qubits(circuit):
 """Add syndrome measurement qubits"""
 # Extend to 5 qubits: 3 data + 2 syndrome
 extended = Circuit(5)
 
 # Copy the data qubit operations
 for gate in circuit.gates:
 extended.gates.append(gate)
 
 # Syndrome measurements
 # Syndrome 1: parity of qubits 0 and 1
 extended.cx(0, 3)
 extended.cx(1, 3)
 
 # Syndrome 2: parity of qubits 1 and 2 
 extended.cx(1, 4)
 extended.cx(2, 4)
 
 return extended
 
 def apply_error(circuit, error_qubit):
 """Apply bit-flip error to specified qubit"""
 if error_qubit is not None and 0 <= error_qubit <= 2:
 circuit.x(error_qubit)
 return circuit
 
 def decode_and_correct(syndrome_results):
 """Determine correction based on syndrome"""
 s1, s2 = syndrome_results
 
 if s1 == 0 and s2 == 0:
 return None, "No error detected"
 elif s1 == 1 and s2 == 0:
 return 0, "Error on qubit 0"
 elif s1 == 1 and s2 == 1:
 return 1, "Error on qubit 1" 
 elif s1 == 0 and s2 == 1:
 return 2, "Error on qubit 2"
 else:
 return None, "Multiple errors (uncorrectable)"
 
 # Test the code
 print("Testing error correction for logical |1:")
 
 for error_pos in [None, 0, 1, 2]:
 print(f"\n--- Test: Error on qubit {error_pos} ---")
 
 # Encode logical |1
 circuit = encode_logical_one()
 
 # Add error
 circuit = apply_error(circuit, error_pos)
 
 # Add syndrome measurement
 circuit_with_syndrome = add_syndrome_qubits(circuit)
 
 print("Circuit with syndrome measurement:")
 print(ASCIIDrawer(circuit_with_syndrome).draw())
 
 # Run and measure syndromes
 state = executor.run(circuit_with_syndrome)
 measurements = state.measure_all(shots=100)
 
 # Extract most common syndrome pattern
 syndrome_counts = {}
 for result, count in measurements.items():
 syndrome = (int(result[3]), int(result[4])) # Syndrome qubits
 syndrome_counts[syndrome] = syndrome_counts.get(syndrome, 0) + count
 
 most_common_syndrome = max(syndrome_counts, key=syndrome_counts.get)
 s1, s2 = most_common_syndrome
 
 # Decode
 correction_qubit, message = decode_and_correct([s1, s2])
 
 print(f"Syndrome: S1={s1}, S2={s2}")
 print(f"Diagnosis: {message}")
 
 if correction_qubit is not None:
 print(f"→ Apply X correction to qubit {correction_qubit}")
 
 # Verify data qubits
 data_results = {}
 for result, count in measurements.items():
 data_bits = result[:3] # First 3 bits are data
 data_results[data_bits] = data_results.get(data_bits, 0) + count
 
 print(f"Data qubit states: {data_results}")
 
 # Check if correction worked
 expected_data = "111" # For logical |1
 if expected_data in data_results and data_results[expected_data] > 50:
 print("SUCCESS: Error correction successful!")
 else:
 print("ERROR: Error correction failed")

three_qubit_bit_flip_code()

## WAVE: Section 2: Noise and Decoherence

Real quantum computers are noisy! Let's explore different types of noise and their effects.

### Types of Quantum Noise
1. **Bit-flip**: |0 ↔ |1 (classical error)
2. **Phase-flip**: |+ ↔ |- (quantum error) 
3. **Depolarizing**: Random Pauli errors
4. **Amplitude damping**: Energy loss (T1 decay)
5. **Dephasing**: Phase randomization (T2 decay)

In [ ]:
def noise_effects_demo():
 """
 Demonstrate different types of quantum noise
 """
 print("WAVE: Quantum Noise and Decoherence")
 print("=" * 35)
 
 def prepare_plus_state():
 """Prepare |+ = (|0 + |1)/√2"""
 circuit = Circuit(1)
 circuit.h(0)
 return circuit
 
 def apply_noise(circuit, noise_type, probability=0.1):
 """Apply different types of noise"""
 # Note: This is a simplified simulation
 # Real noise would be probabilistic and continuous
 
 if noise_type == "bit_flip":
 # Simulate bit-flip with some probability
 circuit.x(0) # Simplified: always apply for demo
 
 elif noise_type == "phase_flip":
 # Phase flip: |+ → |-
 circuit.z(0)
 
 elif noise_type == "depolarizing":
 # Random Pauli operation (simplified)
 circuit.x(0) # Could be X, Y, or Z randomly
 
 elif noise_type == "dephasing":
 # Phase randomization (simplified with Z)
 circuit.z(0)
 
 return circuit
 
 def measure_state_fidelity(circuit, target_state="plus"):
 """Measure how close we are to target state"""
 state = executor.run(circuit)
 measurements = state.measure_all(shots=1000)
 
 if target_state == "plus":
 # For |+, expect equal |0 and |1
 prob_0 = measurements.get('0', 0) / 1000
 prob_1 = measurements.get('1', 0) / 1000
 
 # Ideal |+ has prob_0 = prob_1 = 0.5
 deviation = abs(prob_0 - 0.5) + abs(prob_1 - 0.5)
 fidelity = 1 - deviation
 
 return fidelity, prob_0, prob_1
 
 # Test different noise types
 noise_types = ["bit_flip", "phase_flip", "depolarizing", "dephasing"]
 
 print("Testing noise effects on |+ state:")
 print("=" * 40)
 
 # Perfect case (no noise)
 perfect_circuit = prepare_plus_state()
 fidelity, p0, p1 = measure_state_fidelity(perfect_circuit)
 print(f"No noise: Fidelity={fidelity:.3f}, P(0)={p0:.3f}, P(1)={p1:.3f}")
 
 # With noise
 for noise in noise_types:
 print(f"\n--- {noise.replace('_', ' ').title()} Noise ---")
 
 noisy_circuit = prepare_plus_state()
 noisy_circuit = apply_noise(noisy_circuit, noise)
 
 print("Circuit:")
 print(ASCIIDrawer(noisy_circuit).draw())
 
 fidelity, p0, p1 = measure_state_fidelity(noisy_circuit)
 print(f"Fidelity: {fidelity:.3f}")
 print(f"P(|0): {p0:.3f}, P(|1): {p1:.3f}")
 
 if fidelity < 0.8:
 print("WARNING: Significant degradation due to noise!")
 elif fidelity < 0.95:
 print("WARNING: Noticeable noise effects")
 else:
 print("SUCCESS: Good fidelity maintained")

noise_effects_demo()

## LINK: Section 3: Quantum Entanglement Theory

Entanglement is the most mysterious aspect of quantum mechanics. Let's explore it quantitatively!

### Entanglement Measures
- **Concurrence**: Measure of two-qubit entanglement
- **Von Neumann Entropy**: Information-theoretic measure
- **Negativity**: Entanglement in mixed states

In [ ]:
def entanglement_analysis():
 """
 Analyze quantum entanglement in various states
 """
 print("LINK: Quantum Entanglement Analysis")
 print("=" * 35)
 
 def create_state(state_type):
 """Create different types of two-qubit states"""
 circuit = Circuit(2)
 
 if state_type == "separable":
 # |00 - completely separable
 pass
 
 elif state_type == "bell":
 # |Φ+ = (|00 + |11)/√2 - maximally entangled
 circuit.h(0)
 circuit.cx(0, 1)
 
 elif state_type == "partial":
 # Partially entangled state
 circuit.ry(np.pi/3, 0) # Not fully superposed
 circuit.cx(0, 1)
 
 elif state_type == "ghz":
 # Three-qubit GHZ state (need 3 qubits)
 circuit = Circuit(3)
 circuit.h(0)
 circuit.cx(0, 1)
 circuit.cx(1, 2)
 
 return circuit
 
 def measure_entanglement(circuit):
 """Estimate entanglement through measurement correlations"""
 state = executor.run(circuit)
 measurements = state.measure_all(shots=10000)
 
 if circuit.n_qubits == 2:
 # Two-qubit analysis
 correlations = {}
 total = sum(measurements.values())
 
 for outcome, count in measurements.items():
 prob = count / total
 correlations[outcome] = prob
 
 # Calculate correlation coefficient
 # For Bell states, expect perfect correlation
 prob_00 = correlations.get('00', 0)
 prob_11 = correlations.get('11', 0)
 prob_01 = correlations.get('01', 0)
 prob_10 = correlations.get('10', 0)
 
 # Entanglement witness: |P(00) + P(11) - P(01) - P(10)|
 correlation = abs((prob_00 + prob_11) - (prob_01 + prob_10))
 
 return correlation, correlations
 
 else:
 # Three-qubit analysis
 total = sum(measurements.values())
 prob_000 = measurements.get('000', 0) / total
 prob_111 = measurements.get('111', 0) / total
 
 # GHZ correlation: expect only |000 and |111
 ghz_correlation = prob_000 + prob_111
 
 return ghz_correlation, measurements
 
 # Test different states
 states = ["separable", "bell", "partial"]
 
 for state_type in states:
 print(f"\n--- {state_type.title()} State ---")
 
 circuit = create_state(state_type)
 
 print("Circuit:")
 print(ASCIIDrawer(circuit).draw())
 
 correlation, measurements = measure_entanglement(circuit)
 
 print(f"Entanglement measure: {correlation:.3f}")
 
 # Show measurement distribution
 if isinstance(measurements, dict) and len(measurements) <= 8:
 total = sum(measurements.values()) if state_type != "bell" else 10000
 print("Measurement probabilities:")
 for outcome in sorted(measurements.keys()):
 count = measurements[outcome]
 prob = count / total
 bar = "" * int(prob * 20)
 print(f" |{outcome}: {prob:.3f} {bar}")
 
 # Interpret results
 if correlation > 0.9:
 print("LINK: Highly entangled!")
 elif correlation > 0.5:
 print("LINK: Moderately entangled")
 else:
 print("LINK: Little to no entanglement")
 
 # Special case: Three-qubit GHZ
 print(f"\n--- GHZ State (3 qubits) ---")
 ghz_circuit = create_state("ghz")
 print("Circuit:")
 print(ASCIIDrawer(ghz_circuit).draw())
 
 ghz_correlation, ghz_measurements = measure_entanglement(ghz_circuit)
 print(f"GHZ correlation: {ghz_correlation:.3f}")
 
 if ghz_correlation > 0.8:
 print("LINK: Three-way entanglement achieved!")

entanglement_analysis()

## DEMO: Section 4: Quantum State Tomography

How do you characterize an unknown quantum state? State tomography reconstructs the full quantum state from measurements.

### Process
1. **Prepare identical copies** of the unknown state
2. **Measure in different bases** (X, Y, Z for qubits)
3. **Reconstruct density matrix** from statistics
4. **Verify state properties**

In [ ]:
def quantum_state_tomography():
 """
 Perform quantum state tomography on unknown states
 """
 print("DEMO: Quantum State Tomography")
 print("=" * 30)
 
 def prepare_unknown_state(state_name):
 """Prepare various unknown states to characterize"""
 circuit = Circuit(1)
 
 if state_name == "plus":
 circuit.h(0) # |+ = (|0 + |1)/√2
 
 elif state_name == "minus":
 circuit.x(0)
 circuit.h(0) # |- = (|0 - |1)/√2
 
 elif state_name == "right":
 circuit.h(0)
 circuit.s(0) # |R = (|0 + i|1)/√2
 
 elif state_name == "custom":
 circuit.ry(np.pi/3, 0) # Custom superposition
 
 return circuit
 
 def measure_in_basis(circuit, basis):
 """Measure the state in different bases"""
 # Create a copy of the circuit
 measurement_circuit = Circuit(circuit.n_qubits)
 
 # Copy all gates
 for gate in circuit.gates:
 measurement_circuit.gates.append(gate)
 
 # Apply basis rotation before measurement
 if basis == "X":
 measurement_circuit.h(0) # Rotate to X basis
 elif basis == "Y":
 measurement_circuit.sdg(0) # S† gate
 measurement_circuit.h(0) # Then H for Y basis
 # Z basis needs no rotation
 
 # Measure
 state = executor.run(measurement_circuit)
 measurements = state.measure_all(shots=1000)
 
 prob_0 = measurements.get('0', 0) / 1000
 prob_1 = measurements.get('1', 0) / 1000
 
 return prob_0, prob_1
 
 def reconstruct_bloch_vector(measurements):
 """Reconstruct Bloch vector from measurements"""
 # Extract expectation values
 px0, px1 = measurements['X']
 py0, py1 = measurements['Y'] 
 pz0, pz1 = measurements['Z']
 
 # Calculate expectation values σᵢ = P(0) - P(1)
 x = px0 - px1 # σₓ
 y = py0 - py1 # σᵧ 
 z = pz0 - pz1 # σᵤ
 
 return x, y, z
 
 def analyze_state(bloch_vector, state_name):
 """Analyze the reconstructed state"""
 x, y, z = bloch_vector
 
 print(f"Bloch vector: ({x:.3f}, {y:.3f}, {z:.3f})")
 print(f"State purity: {np.sqrt(x**2 + y**2 + z**2):.3f}")
 
 # Compare with known states
 if state_name == "plus":
 print("Expected for |+: (1, 0, 0)")
 elif state_name == "minus":
 print("Expected for |-: (-1, 0, 0)")
 elif state_name == "right":
 print("Expected for |R: (0, 1, 0)")
 
 # Identify the state
 if abs(x - 1) < 0.1 and abs(y) < 0.1 and abs(z) < 0.1:
 identified = "|+ state"
 elif abs(x + 1) < 0.1 and abs(y) < 0.1 and abs(z) < 0.1:
 identified = "|- state"
 elif abs(x) < 0.1 and abs(y - 1) < 0.1 and abs(z) < 0.1:
 identified = "|R (right) state"
 elif abs(x) < 0.1 and abs(y + 1) < 0.1 and abs(z) < 0.1:
 identified = "|L (left) state"
 elif abs(x) < 0.1 and abs(y) < 0.1 and abs(z - 1) < 0.1:
 identified = "|0 state"
 elif abs(x) < 0.1 and abs(y) < 0.1 and abs(z + 1) < 0.1:
 identified = "|1 state"
 else:
 identified = "Mixed or custom state"
 
 print(f"Identified as: {identified}")
 
 return identified
 
 # Perform tomography on different states
 test_states = ["plus", "minus", "right", "custom"]
 
 for state_name in test_states:
 print(f"\n{'='*50}")
 print(f"DEMO: Tomography of '{state_name}' state")
 print(f"{'='*50}")
 
 # Prepare the unknown state
 unknown_circuit = prepare_unknown_state(state_name)
 
 print("Unknown state preparation:")
 print(ASCIIDrawer(unknown_circuit).draw())
 
 # Measure in all three bases
 measurements = {}
 bases = ['X', 'Y', 'Z']
 
 print("\nMeasurement results:")
 for basis in bases:
 prob_0, prob_1 = measure_in_basis(unknown_circuit, basis)
 measurements[basis] = (prob_0, prob_1)
 
 print(f"{basis} basis: P(0)={prob_0:.3f}, P(1)={prob_1:.3f}")
 
 # Reconstruct the state
 bloch_vector = reconstruct_bloch_vector(measurements)
 
 print(f"\nTARGET: State Reconstruction:")
 identified = analyze_state(bloch_vector, state_name)
 
 print(f"\nSUCCESS: Tomography complete for {state_name} state!")

quantum_state_tomography()

## Section 5: Quantum Supremacy Circuits

Quantum supremacy demonstrations use specially designed circuits that are hard for classical computers to simulate.

### Random Circuit Sampling
- **Goal**: Sample from quantum circuit output distribution
- **Classical difficulty**: Exponential in number of qubits
- **Quantum advantage**: Polynomial time sampling

In [ ]:
def quantum_supremacy_demo():
 """
 Demonstrate quantum supremacy-style random circuit sampling
 """
 print(" Quantum Supremacy Circuit Demonstration")
 print("=" * 47)
 
 def create_random_circuit(n_qubits, depth):
 """Create a random quantum circuit"""
 circuit = Circuit(n_qubits)
 
 # Random single-qubit and two-qubit gates
 gates_1q = ['h', 'x', 'y', 'z', 's', 't']
 gates_2q = ['cx', 'cz']
 
 print(f"Creating random circuit: {n_qubits} qubits, depth {depth}")
 
 for layer in range(depth):
 print(f" Layer {layer + 1}:")
 
 # Random single-qubit gates
 for qubit in range(n_qubits):
 if np.random.random() < 0.7: # 70% chance of gate
 gate = np.random.choice(gates_1q)
 if gate == 'h':
 circuit.h(qubit)
 elif gate == 'x':
 circuit.x(qubit)
 elif gate == 'y':
 circuit.y(qubit)
 elif gate == 'z':
 circuit.z(qubit)
 elif gate == 's':
 circuit.s(qubit)
 elif gate == 't':
 circuit.t(qubit)
 print(f" {gate.upper()} on qubit {qubit}")
 
 # Random two-qubit gates
 if n_qubits > 1:
 for qubit in range(n_qubits - 1):
 if np.random.random() < 0.3: # 30% chance of two-qubit gate
 gate = np.random.choice(gates_2q)
 target = qubit + 1
 
 if gate == 'cx':
 circuit.cx(qubit, target)
 elif gate == 'cz':
 circuit.cz(qubit, target)
 
 print(f" {gate.upper()} on qubits {qubit}-{target}")
 
 return circuit
 
 def analyze_output_distribution(measurements, n_qubits):
 """Analyze the output distribution properties"""
 total_shots = sum(measurements.values())
 n_outcomes = len(measurements)
 max_possible_outcomes = 2**n_qubits
 
 print(f"\nSTATS: Distribution Analysis:")
 print(f"Total shots: {total_shots}")
 print(f"Observed outcomes: {n_outcomes}/{max_possible_outcomes}")
 print(f"Coverage: {n_outcomes/max_possible_outcomes:.3f}")
 
 # Calculate distribution properties
 probabilities = [count/total_shots for count in measurements.values()]
 
 # Uniformity measure (closer to 1/2^n is more uniform)
 ideal_prob = 1/max_possible_outcomes
 uniformity = 1 - np.mean([abs(p - ideal_prob) for p in probabilities])
 
 print(f"Uniformity score: {uniformity:.3f}")
 
 # Entropy (max entropy = n for n qubits)
 entropy = -sum(p * np.log2(p) for p in probabilities if p > 0)
 max_entropy = n_qubits
 normalized_entropy = entropy / max_entropy
 
 print(f"Entropy: {entropy:.3f}/{max_entropy:.3f} ({normalized_entropy:.3f})")
 
 # Show most common outcomes
 sorted_outcomes = sorted(measurements.items(), key=lambda x: x[1], reverse=True)
 print(f"\nTop 5 most common outcomes:")
 for i, (outcome, count) in enumerate(sorted_outcomes[:5]):
 prob = count / total_shots
 decimal = int(outcome, 2)
 print(f" {i+1}. |{outcome} (#{decimal}): {prob:.4f} ({count}/{total_shots})")
 
 return uniformity, normalized_entropy
 
 def classical_simulation_estimate(n_qubits, depth):
 """Estimate classical simulation difficulty"""
 # Very rough estimate of classical simulation cost
 state_vector_size = 2**n_qubits
 operations = depth * n_qubits # Rough estimate
 
 classical_cost = state_vector_size * operations
 
 print(f"\nLAPTOP: Classical Simulation Estimate:")
 print(f"State vector size: 2^{n_qubits} = {state_vector_size:,}")
 print(f"Approximate operations: {operations:,}")
 print(f"Estimated classical cost: {classical_cost:,}")
 
 if classical_cost > 1e12:
 print(" This would be VERY hard for classical computers!")
 elif classical_cost > 1e9:
 print("LIGHTNING: This would be challenging for classical computers")
 else:
 print("LAPTOP: This is manageable for classical computers")
 
 return classical_cost
 
 # Test with different circuit sizes
 test_cases = [
 (3, 3), # Small: 3 qubits, depth 3
 (4, 4), # Medium: 4 qubits, depth 4 
 (5, 3), # Larger: 5 qubits, depth 3
 ]
 
 np.random.seed(42) # For reproducible "randomness"
 
 for n_qubits, depth in test_cases:
 print(f"\n{'='*60}")
 print(f" Test Case: {n_qubits} qubits, depth {depth}")
 print(f"{'='*60}")
 
 # Create random circuit
 random_circuit = create_random_circuit(n_qubits, depth)
 
 print(f"\nTARGET: Final Random Circuit:")
 print(ASCIIDrawer(random_circuit).draw())
 
 # Estimate classical difficulty
 classical_cost = classical_simulation_estimate(n_qubits, depth)
 
 # Run on quantum simulator
 print(f"\nLIGHTNING: Running on quantum simulator...")
 state = executor.run(random_circuit)
 measurements = state.measure_all(shots=5000)
 
 # Analyze results
 uniformity, entropy = analyze_output_distribution(measurements, n_qubits)
 
 # Quantum advantage assessment
 print(f"\nTARGET: Quantum Advantage Assessment:")
 if entropy > 0.8 and uniformity > 0.7:
 print("SUCCESS: High entropy, uniform distribution - good quantum supremacy candidate!")
 elif entropy > 0.6:
 print("LIGHTNING: Moderate entropy - some quantum advantage")
 else:
 print("LAPTOP: Low entropy - limited quantum advantage")
 
 print(f"SUCCESS: Test case complete!\n")

quantum_supremacy_demo()

## AI: Section 6: Quantum Machine Learning

Quantum computing can enhance machine learning through quantum algorithms and quantum data processing.

### Quantum Classification Algorithm
We'll implement a simple quantum classifier using amplitude encoding and measurement.

In [ ]:
def quantum_machine_learning_demo():
 """
 Demonstrate quantum machine learning concepts
 """
 print("AI: Quantum Machine Learning")
 print("=" * 30)
 
 def encode_classical_data(data_point, n_qubits):
 """Encode classical data into quantum amplitudes"""
 circuit = Circuit(n_qubits)
 
 # Normalize data to unit vector
 norm = np.linalg.norm(data_point)
 if norm > 0:
 normalized_data = data_point / norm
 else:
 normalized_data = data_point
 
 # For simplicity, encode data using rotation angles
 # In real QML, would use amplitude encoding
 for i, value in enumerate(normalized_data[:n_qubits]):
 if i < n_qubits:
 angle = np.arcsin(np.clip(abs(value), 0, 1)) * 2
 circuit.ry(angle, i)
 
 return circuit
 
 def quantum_classifier(training_data, test_point):
 """Simple quantum classifier using distance in Hilbert space"""
 print(f"AI: Quantum Classification")
 
 n_qubits = 2 # Use 2 qubits for demonstration
 
 # Encode test point
 test_circuit = encode_classical_data(test_point, n_qubits)
 test_state = executor.run(test_circuit)
 
 print(f"Test point: {test_point}")
 print("Test state encoding:")
 print(ASCIIDrawer(test_circuit).draw())
 
 # Calculate "quantum distances" to training points
 distances = []
 
 for label, train_point in training_data:
 # Encode training point
 train_circuit = encode_classical_data(train_point, n_qubits)
 
 # Create overlap circuit (simplified)
 overlap_circuit = Circuit(n_qubits)
 
 # Encode test point
 for i, value in enumerate(test_point[:n_qubits]):
 angle = np.arcsin(np.clip(abs(value), 0, 1)) * 2
 overlap_circuit.ry(angle, i)
 
 # "Interfere" with training point (simplified)
 for i, value in enumerate(train_point[:n_qubits]):
 angle = np.arcsin(np.clip(abs(value), 0, 1)) * 2
 overlap_circuit.ry(-angle, i) # Negative to "subtract"
 
 # Measure overlap
 overlap_state = executor.run(overlap_circuit)
 measurements = overlap_state.measure_all(shots=1000)
 
 # Distance metric: 1 - P(all zeros)
 prob_all_zeros = measurements.get('0' * n_qubits, 0) / 1000
 distance = 1 - prob_all_zeros
 
 distances.append((distance, label))
 print(f" Distance to {train_point} (label {label}): {distance:.3f}")
 
 # Classify based on minimum distance
 distances.sort()
 predicted_label = distances[0][1]
 
 print(f"\nTARGET: Predicted label: {predicted_label}")
 return predicted_label
 
 def quantum_feature_map(data, n_qubits):
 """Create quantum feature map for data"""
 circuit = Circuit(n_qubits)
 
 # Feature map: encode data through rotations and entanglement
 for i, value in enumerate(data[:n_qubits]):
 angle = value * np.pi # Scale data to rotation angle
 circuit.ry(angle, i)
 
 # Add entanglement between features
 for i in range(n_qubits - 1):
 circuit.cx(i, i + 1)
 
 # Second layer with data-dependent rotations
 for i, value in enumerate(data[:n_qubits]):
 angle = value * value * np.pi # Nonlinear feature
 circuit.rz(angle, i)
 
 return circuit
 
 def variational_quantum_classifier():
 """Demonstrate variational quantum classifier concept"""
 print(f"\nCONFIG: Variational Quantum Classifier")
 
 # Simple 2-class dataset
 data_class_0 = [[0.1, 0.2], [0.2, 0.1], [0.0, 0.3]]
 data_class_1 = [[0.8, 0.9], [0.9, 0.8], [0.7, 1.0]]
 
 # Create feature maps for each class
 n_qubits = 2
 
 print("Class 0 feature maps:")
 for i, data_point in enumerate(data_class_0):
 circuit = quantum_feature_map(data_point, n_qubits)
 print(f" Data {data_point}:")
 print(f" {ASCIIDrawer(circuit).draw()}")
 
 # Measure to see feature distribution
 state = executor.run(circuit)
 measurements = state.measure_all(shots=100)
 prob_00 = measurements.get('00', 0) / 100
 prob_11 = measurements.get('11', 0) / 100
 print(f" P(00)={prob_00:.2f}, P(11)={prob_11:.2f}")
 
 print("\nClass 1 feature maps:")
 for i, data_point in enumerate(data_class_1):
 circuit = quantum_feature_map(data_point, n_qubits)
 print(f" Data {data_point}:")
 print(f" {ASCIIDrawer(circuit).draw()}")
 
 state = executor.run(circuit)
 measurements = state.measure_all(shots=100)
 prob_00 = measurements.get('00', 0) / 100
 prob_11 = measurements.get('11', 0) / 100
 print(f" P(00)={prob_00:.2f}, P(11)={prob_11:.2f}")
 
 # Demo 1: Simple quantum classifier
 print("Demo 1: Quantum k-NN Classifier")
 print("=" * 40)
 
 # Training data: (label, [feature1, feature2])
 training_data = [
 (0, [0.1, 0.2]), # Class 0
 (0, [0.2, 0.1]), # Class 0
 (1, [0.8, 0.9]), # Class 1
 (1, [0.9, 0.8]), # Class 1
 ]
 
 # Test points
 test_points = [
 [0.15, 0.15], # Should be class 0
 [0.85, 0.85], # Should be class 1
 [0.5, 0.5], # Boundary case
 ]
 
 for test_point in test_points:
 predicted = quantum_classifier(training_data, test_point)
 print(f"Test point {test_point} → Class {predicted}\n")
 
 # Demo 2: Variational quantum classifier
 variational_quantum_classifier()
 
 print("\nSUCCESS: Quantum Machine Learning Demo Complete!")
 print("\nKey QML Concepts Demonstrated:")
 print("SUCCESS: Amplitude encoding of classical data")
 print("SUCCESS: Quantum feature maps")
 print("SUCCESS: Quantum distance measures")
 print("SUCCESS: Variational quantum circuits")

quantum_machine_learning_demo()

## SUCCESS: Advanced Concepts Mastered!

Congratulations! You've now explored the most advanced topics in quantum computing:

### TARGET: What You've Learned

1. **SECURITY: Quantum Error Correction**: How to protect quantum information from noise
2. **WAVE: Noise Models**: Understanding real-world quantum decoherence 
3. **LINK: Entanglement Theory**: Quantifying quantum correlations
4. **DEMO: State Tomography**: Reconstructing unknown quantum states
5. ** Quantum Supremacy**: Circuits that challenge classical computers
6. **AI: Quantum Machine Learning**: AI enhanced by quantum computing

### Next Steps in Your Quantum Journey

**Research Directions:**
- DOCS: **Quantum Chemistry**: Simulating molecular systems
- LAPTOP: **Quantum Optimization**: QAOA and variational algorithms
- **Quantum Cryptography**: Secure communication protocols
- INTERACTIVE: **Quantum Games**: Game theory with quantum strategies

**Practical Applications:**
- INDUSTRY: **Industry Problems**: Apply quantum algorithms to real challenges
- SCIENCE: **Research Projects**: Contribute to quantum computing research
- EDUCATION: **Teaching**: Share quantum knowledge with others
- **Career Development**: Join the quantum computing industry

### SPARKLE: Keep Exploring!

The quantum world is vast and full of mysteries. Use the tools and concepts you've learned here to:

- **Build your own quantum algorithms**
- **Explore quantum hardware platforms** 
- **Contribute to open-source quantum software**
- **Join the quantum computing community**

## EXPERIMENT: Final Challenge

Create your own advanced quantum algorithm in the cell below. Combine concepts from multiple sections to build something novel!

In [ ]:
# EXPERIMENT: Your Advanced Quantum Algorithm
# Combine concepts from this notebook to create something new!

def my_advanced_quantum_algorithm():
 """
 Design your own advanced quantum algorithm here!
 
 Ideas to combine:
 - Error correction + quantum algorithms
 - Entanglement + machine learning 
 - Tomography + noise characterization
 - Supremacy circuits + practical applications
 """
 print("EXPERIMENT: My Advanced Quantum Algorithm")
 
 # Example: Entanglement-enhanced quantum sensing
 circuit = Circuit(3)
 
 # Create entangled sensor network
 circuit.h(0)
 circuit.cx(0, 1)
 circuit.cx(1, 2) # GHZ state for enhanced sensitivity
 
 # Apply "signal" to measure (simulated)
 signal_strength = 0.1
 circuit.rz(signal_strength, 0) # Phase signal on first qubit
 
 print("Entangled quantum sensor:")
 print(ASCIIDrawer(circuit).draw())
 
 # Measure correlations to detect signal
 state = executor.run(circuit)
 measurements = state.measure_all(shots=1000)
 
 # Analyze sensitivity enhancement
 correlation_000_111 = (measurements.get('000', 0) + measurements.get('111', 0)) / 1000
 print(f"Entangled correlation: {correlation_000_111:.3f}")
 print(f"Signal detected with enhanced quantum sensitivity!")
 
 return measurements

# Run your algorithm!
my_advanced_quantum_algorithm()

print("\nSUCCESS: Congratulations on mastering advanced quantum computing!")
print("The quantum future is in your hands! SHINE: ")